In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import sys, os, time, collections

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'cells'))
# sys.path.append(os.path.join(os.environ['REPO_DIR'], 'annotation'))
# sys.path.append(os.path.join(os.environ['REPO_DIR']bb, 'learning'))

import numpy as np
import scipy
import skimage
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from skimage.measure import regionprops, label, find_contours
from skimage.color import rgb2hsv
from scipy import ndimage as ndi
import pandas
from scipy.signal import argrelmax
from scipy.stats import linregress

from annotation_utilities import *
from registration_utilities import *
from learning_utilities import *
from data_manager import *
from utilities2015 import *
from cell_utilities import *
from metadata import *
from distributed_utilities import download_from_s3

from xj_utilities import *

/shared/MouseBrainAtlasXiang/utilities/utilities2015.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist

Setting environment for AWS compute node


Cannot import mxnet.


In [2]:
def fun_get_regionprop(skimage_measure_region, prop):
    """skimage_measure_region: regionprops, returned by skimage.measure.regionprps(), of the type skimage.measure.regionprops
        prop: region property, can be built-in properties like 'area', 'moments_hu', eccentricity', or in addition, 'compactness'
    Returns: regionprops[prop] in float/numpy.ndarray/int, depends on the input.  """
    buildinProps = ['coords','centroid','orientation', 'eccentricity','area','orientation','moments_hu','bbox','equivalent_diameter','label','local_centroid','major_axis_length','solidity','minor_axis_length','perimeter','solidity']
    PI = 3.1415926
    if prop in buildinProps:
        return skimage_measure_region[prop]
    elif prop == 'compactness':
        return float(skimage_measure_region['perimeter'])**2 / float(skimage_measure_region['area']) / (4 * PI)

def fun_get_regionprop_list(skimage_measure_region_list, prop):
    regionprop_list = [fun_get_regionprop(record, prop) for record in skimage_measure_region_list]
    return np.array(regionprop_list)

def fun_save_typical_blobs_info(typical_blobs, prop_to_save, stack, section, dataType='typical',dataFolderName=None):
    # Remove duplicated blobs and save tpical blob information.
    tempTBID = 0
    tempBID = -1
    unique_BID = []
    for record in typical_blobs:
        if tempBID != record[1]:
            tempBID = record[1]
            unique_BID.append(tempTBID)
        tempTBID = tempTBID + 1
    unique_BID = np.array(unique_BID)
    typical_blob_regionprops = np.array([record[2] for record in typical_blobs])[unique_BID]
    fun_save_regionprops(regionprop_List=typical_blob_regionprops,prop_to_save=prop_to_save,stack=stack,sec=section,dataType=dataType, dataFolderName=dataFolderName)

def fun_get_matched_blobs_IDs(matched_paris):
    ''' matched_pairs: a list of [section, blobID, typical_blob_props, scan_sec, matched_blob, matched_blob_props, sim_matrix]
        return: 
            matched_blobs_from_section: a list of sections indicates where do the matched blobs come from
            matched_blobs_with_typicalBlob: a list of typical_blob_ID indicates which typical blob is matched'''
    matched_blobs_from_section = []
    matched_blobs_with_typicalBlob = []
    typical_blobs_id = -1
    blobID = -1
    for record in matched_paris:
        if blobID != record[1]:
            blobID = record[1]
            typical_blobs_id +=1
        temp_matched_blobs_section = record[3]
        temp_num_matched_blobs = len(record[4])
        matched_blobs_from_section += [temp_matched_blobs_section] * temp_num_matched_blobs
        matched_blobs_with_typicalBlob += [typical_blobs_id] * temp_num_matched_blobs
    return matched_blobs_from_section, matched_blobs_with_typicalBlob    
    
    
def fun_save_matched_blobs_info(matched_paris, prop_to_save, stack, section, dataType='matched', dataFolderName=None):
    matched_blob_regionprops = [record[5] for record in matched_paris]
    matched_blob_regionprops = np.concatenate(tuple(matched_blob_regionprops)).tolist()
    fun_save_regionprops(regionprop_List=matched_blob_regionprops, prop_to_save=prop_to_save, stack=stack, sec=section, dataType=dataType, dataFolderName=dataFolderName)
    
    matched_blobs_from_section, matched_blobs_with_typicalBlob = fun_get_matched_blobs_IDs(matched_paris=matched_paris)
    matched_blobs_from_section = np.array(matched_blobs_from_section, dtype=np.int16)
    matched_blobs_with_typicalBlob = np.array(matched_blobs_with_typicalBlob, dtype=np.int16)
    bp.pack_ndarray_file(matched_blobs_with_typicalBlob, get_typical_cell_data_filepath(what='TBID', stack=stack, sec=section, dataType=dataType, dataFolderName=dataFolderName) )
    bp.pack_ndarray_file(matched_blobs_from_section, get_typical_cell_data_filepath(what='section', stack=stack, sec=section, dataType=dataType, dataFolderName=dataFolderName) )
   

In [3]:
def fun_save_regionprops(regionprop_List, prop_to_save, stack, sec, dataType='typical', dataFolderName=None):
    for tempProp in prop_to_save:
        tempProp_data = fun_get_regionprop_list(skimage_measure_region_list=regionprop_List, prop=tempProp)
        
        if tempProp == 'coords':
            tempProp_data = map(lambda data: np.array(data, dtype=np.int16), tempProp_data)
        elif tempProp == 'moments_hu':
            tempProp_data = map(lambda data: np.array(data, dtype=np.float32), tempProp_data)
            tempProp_data = np.row_stack(tuple(tempProp_data))
        elif tempProp == 'centroid':
            tempProp_data = map(lambda data: np.array(data, dtype=np.float32), tempProp_data)
            tempProp_data = np.row_stack(tuple(tempProp_data))
        elif tempProp == 'area':
            tempProp_data = np.array(tempProp_data, np.int32)
        elif tempProp == 'eccentricity':
            tempProp_data = np.array(tempProp_data, np.float32)
        elif tempProp == 'equivalent_diameter':
            tempProp_data = np.array(tempProp_data, np.float32)
        elif tempProp == 'compactness':
            tempProp_data = np.array(tempProp_data, np.float32)
        elif tempProp == 'perimeter':
            tempProp_data = np.array(tempProp_data, np.int32)
        elif tempProp == 'solidity':
            tempProp_data = np.array(tempProp_data, np.float32)
        tempFp = get_typical_cell_data_filepath(what=tempProp,stack=stack,sec=sec, dataType=dataType, dataFolderName=dataFolderName)
        create_if_not_exists(os.path.dirname(tempFp))
        if tempFp.endswith('.hdf'):
            save_hdf_v2(tempProp_data, fn=tempFp)
        elif tempFp.endswith('.bp'):
            bp.pack_ndarray_file(tempProp_data, tempFp)
        else:
            print 'Unrecognized data type. Save failed.'
    return 0

In [4]:
scan_parameters = {}

In [5]:
scan_parameters['dataFolderName'] = 'OnlyArea'
scan_parameters['stack'] = 'MD589'
scan_parameters['patch_size'] = 448
scan_parameters['patch_half_size'] = scan_parameters['patch_size']/2
scan_parameters['stride'] = 112

scan_parameters['section_limits'] = metadata_cache['section_limits'][scan_parameters['stack']]
scan_parameters['o_crop'] = True
scan_parameters['o_clear_border'] = True
scan_parameters['o_relabel'] = True
scan_parameters['oriImL1'], scan_parameters['oriImL0'] = metadata_cache['image_shape'][scan_parameters['stack']]
scan_parameters['scan_section_range'] = 1
scan_parameters['prop'] = ['area','centroid']
# scan_parameters['prop'] = ['centroid','eccentricity','area','orientation','moments_hu','bbox','equivalent_diameter','perimeter','compactness','label','major_axis_length','minor_axis_length']
scan_parameters['prop_for_comparison'] = ['area']
if 'moments_hu' in scan_parameters['prop_for_comparison']:
    scan_parameters['compare_weight'] = [1 for i in range(6 + len(scan_parameters['prop_for_comparison']))]
else:
    scan_parameters['compare_weight'] = [1 for i in range(len(scan_parameters['prop_for_comparison']))]
scan_parameters['compare_weight'] = np.array(scan_parameters['compare_weight'])/ float(np.sum(scan_parameters['compare_weight']))
scan_parameters['similarity_threshold'] = 0.9
scan_parameters['false_similarity_threshold'] = 0.7
scan_parameters['o_fix_scan_size'] = True
scan_parameters['scan_size'] = 112
scan_parameters['scan_size_coeff'] = 5
scan_parameters['builtInProps'] = ['centroid','orientation', 'eccentricity','area','moments_hu','bbox','equivalent_diameter','label','local_centroid','major_axis_length','minor_axis_length','perimeter','solidity']
scan_parameters['prop_to_save'] = ['moments_hu','centroid','area','eccentricity','equivalent_diameter', 'compactness', 'orientation','solidity']
scan_parameters['scan_outside_margin'] = 3000
scan_parameters['false_match_sample_num'] = 200
stack = scan_parameters['stack']

In [6]:
contour_df = DataManager.load_annotation_v4(stack=scan_parameters['stack'],by_human=True,suffix='contours',timestamp='latest')
contour_df = convert_annotation_v3_original_to_aligned_cropped(contour_df=contour_df,stack=stack)
structure_name = '7N'
vertice_7N_dic = {record['section']:np.array(record['vertices'],np.int) for _, record in contour_df[contour_df['name']==structure_name].iterrows()}
sectionList_7N = np.sort(vertice_7N_dic.keys())

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD589" "/shared/CSHL_labelings_v3/MD589" --exclude "*" --include "*contours*"
latest timestamp:  10042017100807


In [7]:
# Whole image setting:
scan_parameters['crop_0_min'] = 0
scan_parameters['crop_1_min'] = 0
scan_parameters['crop_0_max'] = scan_parameters['oriImL0']
scan_parameters['crop_1_max'] = scan_parameters['oriImL1']
scan_parameters['crop_range_mmxx'] = (scan_parameters['crop_0_min'], scan_parameters['crop_1_min'], scan_parameters['crop_0_max'], scan_parameters['crop_1_max'])
scan_parameters['crop_range_mxmx'] = fun_mmxx_to_mxmx(*scan_parameters['crop_range_mmxx'])
scan_parameters['im0max'] = scan_parameters['crop_0_max'] - scan_parameters['crop_0_min']
scan_parameters['im1max'] = scan_parameters['crop_1_max'] - scan_parameters['crop_1_min']

In [8]:
# margin = 10
# bboxs_7N = {sec:fun_polygon_bbox(vertice_7N_dic[sec]) for sec in sectionList_7N}
# bbox_7N = fun_polygons_bbox(bboxs_7N.values(),margin=margin)
# # bbox_7N = bboxs_7N[154]
# scan_parameters['crop_range_mmxx'] = bbox_7N
# scan_parameters['crop_0_min'], scan_parameters['crop_1_min'],scan_parameters['crop_0_max'],scan_parameters['crop_1_max'] = bbox_7N 
# scan_parameters['im0max'] = scan_parameters['crop_0_max'] - scan_parameters['crop_0_min']
# scan_parameters['im1max'] = scan_parameters['crop_1_max'] - scan_parameters['crop_1_min']
# scan_parameters['crop_range_mxmx'] = fun_mmxx_to_mxmx(*scan_parameters['crop_range_mmxx'])

In [9]:
# Get avaliable sections:
stack = scan_parameters['stack']
section_begin, section_end = scan_parameters['section_limits']
# load_section_num = 100
# load_start_section = 152 - section_begin
all_valid_section = fun_get_valid_section_list(stack=stack)
# secList = all_valid_section[load_start_section:load_section_num + load_start_section]
# secList = list(set(all_valid_section).intersection(sectionList_7N))
secList = all_valid_section
scan_parameters['secList'] = secList

In [10]:
# # loading data
# image = {}
# for tempSec in secList:
#     try:
#         image[tempSec] = fun_crop_images(DataManager.load_image_v2(stack=stack, section=tempSec, version='jpeg', prep_id=2),*scan_parameters['crop_range_mmxx'],im0max=scan_parameters['oriImL0'],im1max=scan_parameters['oriImL1'] )
#     except:
#         sys.stderr.write('Cannot import section %d...\n'%tempSec)
#         continue

In [11]:
def fun_collect_blobs(section, scan_parameters, collectFlaseMatched=False, saveMatched=True):
    scan_range = scan_parameters['scan_section_range']
    im0max = scan_parameters['im0max']
    im1max = scan_parameters['im1max']
    prop = scan_parameters['prop']
    prop_for_comparison = scan_parameters['prop_for_comparison']
    compare_weight = scan_parameters['compare_weight']
    o_simil_threshold = scan_parameters['similarity_threshold']
    o_false_simil_threshold = scan_parameters['false_similarity_threshold'] # Option. Default is equal to o_simil_threshold
    o_fix_scan_size = scan_parameters['o_fix_scan_size']
    o_scan_size_coeff = scan_parameters['scan_size_coeff']
    o_scan_size = scan_parameters['scan_size']  
    secList = scan_parameters['secList'] 
    stack = scan_parameters['stack']
    dataFolderName = scan_parameters['dataFolderName']
    # data_typical_blobs = {}
    # data_matched_paris = {}
    # data_false_typical_blobs = {}
    # data_false_matched_pairs = {}
    # data_blob_prop_dic = {}
    # data_blob_idx_selection = {}
#     section = 154
    # for section in secList:
    typical_blobs = []
    matched_paris = []
    false_typical_blobs = []
    false_matched_paris = []
    cell_centroids = {}
    cell_numbers = {}
    cell_global_coord = {}
    im_blob_prop = {}
    im_label = {}
    im_BW = {}
    sec_load_data_list = range(section - scan_range, section + scan_range + 1)
    scan_section = list(sec_load_data_list)
    scan_section.remove(section)
    ### loading data and reconstructed labeled images ###
    tempList = list(sec_load_data_list)
    for tempSec in tempList:
        if tempSec in secList:
            cell_global_coord[tempSec] = load_cell_data('coords', stack=stack, sec=tempSec)
            _, temp_im_blob_prop, _ = fun_reconstruct_labeled_image(cell_global_coord[tempSec],crop_range= scan_parameters['crop_range_mxmx'], 
                                                                        oriImL0=scan_parameters['oriImL0'],oriImL1=scan_parameters['oriImL1'])
#             im_label[tempSec] = temp_im_label
#             im_BW[tempSec] = temp_im_label > 0
            im_blob_prop[tempSec] = np.array(temp_im_blob_prop)
        else:
            sys.stderr.write('Warning: missing section %d'%tempSec)
            scan_section.remove(tempSec)
            sec_load_data_list.remove(tempSec)

    ### Start getting region properties of each blob    ###
    blob_prop_dic = fun_regionprops_dic(im_blob_prop=im_blob_prop,scan_parameters=scan_parameters)
    # data_blob_prop_dic[section] = blob_prop_dic[section] # Save data
    blob_idx_selection = {tempSec : np.logical_and.reduce(np.row_stack(
        (
        #      blob_prop_dic[tempSec]['compactness']<2.5,
         blob_prop_dic[tempSec]['area']<3000,
         blob_prop_dic[tempSec]['area']>200))) for tempSec in sec_load_data_list}  

    # data_blob_idx_selection[section] = blob_idx_selection[section] # Save data

    n_blobs = {tempSec: len(im_blob_prop[tempSec]) for tempSec in im_blob_prop.keys()}
    secList_in_BlobPropDic = im_blob_prop.keys()
    if set(scan_section).issubset(set(secList_in_BlobPropDic)):
        pass
    else:
        print('Warrning: Scaned section(s) not included in input im_blob_prop')



    ### Start scanning ###
    for blobID in range(n_blobs[section]):
        if (blobID % 1000 == 0):
            print('Section %d Finished percentage: %f'%(section, (float(blobID)*100 / n_blobs[section]) ))

        temp_curr_blob_props = {}
        for tempProp in prop:
            temp_curr_blob_props[tempProp] = blob_prop_dic[section][tempProp][blobID]
        if not blob_idx_selection[section][blobID]:
    #         print('Blob %d eliminated'%blobID)
            continue

        tempB1_idx_loc = temp_curr_blob_props['centroid']
        if o_fix_scan_size:
            temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max,o_form='2D')
            temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max)
        else:
            temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max,o_form='2D')
            temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max)


        for tempSec in scan_section:
    #         print('Blbo %d left. Start scanning'%blobID)
            if tempSec not in secList_in_BlobPropDic:
                continue

            # Find blobs at the nearby location in the scaned section
            tempBlobInside = fun_blobs_in_polygen(blob_prop_dic[tempSec]['centroid'],temp_next_sec_range,coor_order='cr')
            tempBlobInsideIndex = np.where(tempBlobInside)[0]
            temp_num_blob = len(tempBlobInsideIndex)
            if collectFlaseMatched == True:
                tempBlobOutside = fun_blobs_out_polygen(blob_prop_dic[tempSec]['centroid'], temp_next_sec_range, coor_order='cr', margin=scan_parameters['scan_outside_margin'])
                tempBlobOutsideIndex = np.where(tempBlobOutside)[0]
                tempBlobOutsideIndex = np.random.choice(tempBlobOutsideIndex, size=scan_parameters['false_match_sample_num'], replace=False)
                temp_num_outblob = len(tempBlobOutsideIndex)

            ### Typical blobs###

            if temp_num_blob:
                temp_sim = {}
                for temp_prop in prop_for_comparison:
                          temp_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobInside],distance_type=temp_prop))
                temp_sim_matrix = np.column_stack((temp_sim[temp_prop] for temp_prop in prop_for_comparison))

                #### Blob comparison ####
                temp_weighted_sim = np.dot(temp_sim_matrix,compare_weight)
                temp_compare_result = temp_weighted_sim > o_simil_threshold
                if any(temp_compare_result.tolist()):
                    typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                    matched_paris.append([section,blobID,
                                          im_blob_prop[section][blobID],
                                          tempSec,
                                          tempBlobInsideIndex[temp_compare_result],
                                          im_blob_prop[tempSec][tempBlobInsideIndex[temp_compare_result]],
                                          temp_sim_matrix[temp_compare_result,:]])
            else:
                pass

            ### False-typical blobs###
            if collectFlaseMatched == True:
                if temp_num_outblob:
                    temp_false_sim = {}
                    for temp_prop in prop_for_comparison:
                              temp_false_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobOutsideIndex],distance_type=temp_prop))
                    temp_false_sim_matrix = np.column_stack((temp_false_sim[temp_prop] for temp_prop in prop_for_comparison))

                    #### Blob comparison ####
                    temp_weighted_false_sim = np.dot(temp_false_sim_matrix,compare_weight)
                    temp_false_compare_result = temp_weighted_false_sim > o_false_simil_threshold
                    if any(temp_false_compare_result.tolist()):
                        false_typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                        false_matched_paris.append([section,blobID,im_blob_prop[section][blobID],
                                                    tempSec, 
                                                    tempBlobOutsideIndex[temp_false_compare_result], 
                                                    im_blob_prop[tempSec][tempBlobOutsideIndex[temp_false_compare_result]],
                                                    temp_false_sim_matrix[temp_false_compare_result,:]])
                else:
                    pass
    tempFp = get_typical_cell_data_filepath(what='scan_parameters',stack=stack,sec=section,dataFolderName=dataFolderName)
    create_if_not_exists(os.path.dirname(tempFp))
    save_pickle(scan_parameters,tempFp)
    fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='typical',dataFolderName=dataFolderName)
    if saveMatched == True:
        fun_save_matched_blobs_info(matched_paris=matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='matched',dataFolderName=dataFolderName)
    if collectFlaseMatched == True:
        fun_save_typical_blobs_info(typical_blobs=false_typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_typical',dataFolderName=dataFolderName)
        fun_save_matched_blobs_info(matched_paris=false_matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_matched',dataFolderName=dataFolderName)
    print('Section %d result saved.'% section)
    return 0
# data_typical_blobs[section] = typical_blobs
# data_matched_paris[section] = matched_paris
# data_false_typical_blobs[section] = false_typical_blobs
# data_false_matched_pairs[section] = false_matched_paris   

In [11]:
from multiprocess import Pool

In [9]:
scan_parameters['dataFolderName'] = 'NoCoors_FST07'
scan_parameters['scan_outside_margin'] = 3000
scan_parameters['false_match_sample_num'] = 200

In [10]:
fun_collect_blobs(section=92, scan_parameters=scan_parameters, collectFlaseMatched=False, saveMatched=False)

Section 92 Finished percentage: 0.000000
Section 92 Finished percentage: 1.916847
Section 92 Finished percentage: 3.833694
Section 92 Finished percentage: 5.750542
Section 92 Finished percentage: 7.667389
Section 92 Finished percentage: 9.584236
Section 92 Finished percentage: 11.501083
Section 92 Finished percentage: 13.417930
Section 92 Finished percentage: 15.334777
Section 92 Finished percentage: 17.251625
Section 92 Finished percentage: 19.168472
Section 92 Finished percentage: 21.085319
Section 92 Finished percentage: 23.002166
Section 92 Finished percentage: 24.919013
Section 92 Finished percentage: 26.835860
Section 92 Finished percentage: 28.752708
Section 92 Finished percentage: 30.669555
Section 92 Finished percentage: 32.586402
Section 92 Finished percentage: 34.503249
Section 92 Finished percentage: 36.420096
Section 92 Finished percentage: 38.336943
Section 92 Finished percentage: 40.253791
Section 92 Finished percentage: 42.170638
Section 92 Finished percentage: 44.08748

0

In [ ]:
# To save the coords, only 4 process can be parallelized to avoid 
pool = Pool(4)
pool.map(lambda sec: fun_collect_blobs(section=sec, scan_parameters=scan_parameters, collectFlaseMatched=True, saveMatched=True), secList)

Section 92 Finished percentage: 0.000000
Section 92 Finished percentage: 1.916847
Section 92 Finished percentage: 3.833694
Section 92 Finished percentage: 5.750542
Section 92 Finished percentage: 7.667389
Section 92 Finished percentage: 9.584236
Section 92 Finished percentage: 11.501083
Section 92 Finished percentage: 13.417930
Section 92 Finished percentage: 15.334777
Section 92 Finished percentage: 17.251625
Section 129 Finished percentage: 0.000000
Section 92 Finished percentage: 19.168472
Section 92 Finished percentage: 21.085319
Section 92 Finished percentage: 23.002166
Section 129 Finished percentage: 1.817389
Section 110 Finished percentage: 0.000000
Section 92 Finished percentage: 24.919013
Section 92 Finished percentage: 26.835860
Section 129 Finished percentage: 3.634778
Section 110 Finished percentage: 1.859324
Section 92 Finished percentage: 28.752708
Section 92 Finished percentage: 30.669555
Section 129 Finished percentage: 5.452166
Section 92 Finished percentage: 32.58640

Section 147 Finished percentage: 53.342026
Section 110 Finished percentage: 96.684826
Section 147 Finished percentage: 54.823749
Section 129 Finished percentage: 94.504216
Section 110 Finished percentage: 98.544150
Section 147 Finished percentage: 56.305472
Section 129 Finished percentage: 96.321605
Section 147 Finished percentage: 57.787195
Section 129 Finished percentage: 98.138994
Section 147 Finished percentage: 59.268918
Section 129 Finished percentage: 99.956383
Section 147 Finished percentage: 60.750641
Section 147 Finished percentage: 62.232364
Section 147 Finished percentage: 63.714087
Section 147 Finished percentage: 65.195810
Section 147 Finished percentage: 66.677533
Section 147 Finished percentage: 68.159256
Section 147 Finished percentage: 69.640979
Section 147 Finished percentage: 71.122701
Section 147 Finished percentage: 72.604424
Section 147 Finished percentage: 74.086147
Section 147 Finished percentage: 75.567870
Section 147 Finished percentage: 77.049593
Section 147

Section 130 Finished percentage: 82.481029
Section 130 Finished percentage: 84.130650
Section 130 Finished percentage: 85.780271
Section 130 Finished percentage: 87.429891
Section 147 result saved.
Section 130 Finished percentage: 89.079512
Section 130 Finished percentage: 90.729132
Section 130 Finished percentage: 92.378753
Section 130 Finished percentage: 94.028373
Section 130 Finished percentage: 95.677994
Section 130 Finished percentage: 97.327615
Section 130 Finished percentage: 98.977235
Section 148 Finished percentage: 0.000000
Section 148 Finished percentage: 1.394156
Section 148 Finished percentage: 2.788311
Section 148 Finished percentage: 4.182467
Section 148 Finished percentage: 5.576623
Section 148 Finished percentage: 6.970778
Section 148 Finished percentage: 8.364934
Section 148 Finished percentage: 9.759090
Section 148 Finished percentage: 11.153246
Section 148 Finished percentage: 12.547401
Section 148 Finished percentage: 13.941557
Section 148 Finished percentage: 15.

Section 131 Finished percentage: 61.826820
Section 94 Finished percentage: 19.082149
Section 94 Finished percentage: 20.990364
Section 131 Finished percentage: 63.453841
Section 131 Finished percentage: 65.080863
Section 94 Finished percentage: 22.898578
Section 131 Finished percentage: 66.707885
Section 94 Finished percentage: 24.806793
Section 131 Finished percentage: 68.334906
Section 94 Finished percentage: 26.715008
Section 131 Finished percentage: 69.961928
Section 94 Finished percentage: 28.623223
Section 131 Finished percentage: 71.588949
Section 94 Finished percentage: 30.531438
Section 131 Finished percentage: 73.215971
Section 94 Finished percentage: 32.439653
Section 131 Finished percentage: 74.842992
Section 94 Finished percentage: 34.347868
Section 131 Finished percentage: 76.470014
Section 94 Finished percentage: 36.256082
Section 131 Finished percentage: 78.097036
Section 94 Finished percentage: 38.164297
Section 94 Finished percentage: 40.072512
Section 131 Finished pe

Section 113 Finished percentage: 97.952244
Section 149 Finished percentage: 64.640246
Section 113 Finished percentage: 99.800399
Section 132 Finished percentage: 47.528517
Section 149 Finished percentage: 66.179300
Section 132 Finished percentage: 49.167431
Section 149 Finished percentage: 67.718353
Section 132 Finished percentage: 50.806346
Section 149 Finished percentage: 69.257407
Section 132 Finished percentage: 52.445260
Section 149 Finished percentage: 70.796460
Section 132 Finished percentage: 54.084175
Section 149 Finished percentage: 72.335514
Section 149 Finished percentage: 73.874567
Section 132 Finished percentage: 55.723089
Section 149 Finished percentage: 75.413621
Section 132 Finished percentage: 57.362003
Section 149 Finished percentage: 76.952674
Section 132 Finished percentage: 59.000918
Section 149 Finished percentage: 78.491728
Section 132 Finished percentage: 60.639832
Section 149 Finished percentage: 80.030781
Section 132 Finished percentage: 62.278747
Section 149

...

Section 241 Finished percentage: 75.633932
Section 200 Finished percentage: 77.498122
Section 235 Finished percentage: 87.844108
Section 200 Finished percentage: 78.690401
Section 241 Finished percentage: 76.960843
Section 235 Finished percentage: 89.081349
Section 200 Finished percentage: 79.882680
Section 241 Finished percentage: 78.287754
Section 235 Finished percentage: 90.318590
Section 200 Finished percentage: 81.074959
Section 200 Finished percentage: 82.267237
Section 241 Finished percentage: 79.614665
Section 235 Finished percentage: 91.555830
Section 241 Finished percentage: 80.941576
Section 200 Finished percentage: 83.459516
Section 235 Finished percentage: 92.793071
Section 241 Finished percentage: 82.268487
Section 200 Finished percentage: 84.651795
Section 235 Finished percentage: 94.030312
Section 241 Finished percentage: 83.595398
Section 200 Finished percentage: 85.844074
Section 200 Finished percentage: 87.036353
Section 241 Finished percentage: 84.922309
Section 235

Section 236 Finished percentage: 24.660912
Section 242 Finished percentage: 55.748181
Section 256 Finished percentage: 4.590947
Section 216 Finished percentage: 98.918405
Section 236 Finished percentage: 25.893958
Section 242 Finished percentage: 57.107893
Section 236 Finished percentage: 27.127004
Section 242 Finished percentage: 58.467605
Section 256 Finished percentage: 5.738683
Section 236 Finished percentage: 28.360049
Section 242 Finished percentage: 59.827317
Section 236 Finished percentage: 29.593095
Section 256 Finished percentage: 6.886420
Section 242 Finished percentage: 61.187028
Section 236 Finished percentage: 30.826141
Section 242 Finished percentage: 62.546740
Section 236 Finished percentage: 32.059186
Section 256 Finished percentage: 8.034157
Section 242 Finished percentage: 63.906452
Section 236 Finished percentage: 33.292232
Section 242 Finished percentage: 65.266164
Section 256 Finished percentage: 9.181893
Section 236 Finished percentage: 34.525277
Section 242 Fini

Section 217 Finished percentage: 14.702541
Section 256 Finished percentage: 99.853090
Section 217 Finished percentage: 16.039135
Section 217 Finished percentage: 17.375730
Section 217 Finished percentage: 18.712325
Section 217 Finished percentage: 20.048919
Section 217 Finished percentage: 21.385514
Section 217 Finished percentage: 22.722109
Section 217 Finished percentage: 24.058703
Section 217 Finished percentage: 25.395298
Section 217 Finished percentage: 26.731892
Section 217 Finished percentage: 28.068487
Section 217 Finished percentage: 29.405082
Section 217 Finished percentage: 30.741676
Section 217 Finished percentage: 32.078271
Section 217 Finished percentage: 33.414866
Section 217 Finished percentage: 34.751460
Section 243 Finished percentage: 0.000000
Section 217 Finished percentage: 36.088055
Section 243 Finished percentage: 1.233867
Section 217 Finished percentage: 37.424649
Section 243 Finished percentage: 2.467734
Section 243 Finished percentage: 3.701602
Section 217 Fin

Section 237 Finished percentage: 53.424641
Section 237 Finished percentage: 54.638837
Section 237 Finished percentage: 55.853034
Section 237 Finished percentage: 57.067230
Section 237 Finished percentage: 58.281426
Section 237 Finished percentage: 59.495623
Section 237 Finished percentage: 60.709819
Section 237 Finished percentage: 61.924016
Section 237 Finished percentage: 63.138212
Section 237 Finished percentage: 64.352408
Section 237 Finished percentage: 65.566605
Section 237 Finished percentage: 66.780801
Section 237 Finished percentage: 67.994998
Section 237 Finished percentage: 69.209194
Section 237 Finished percentage: 70.423390
Section 237 Finished percentage: 71.637587
Section 237 Finished percentage: 72.851783
Section 237 Finished percentage: 74.065979
Section 237 Finished percentage: 75.280176
Section 237 Finished percentage: 76.494372
Section 237 Finished percentage: 77.708569
Section 237 Finished percentage: 78.922765
Section 237 Finished percentage: 80.136961
Section 237

Section 244 Finished percentage: 81.523753
Section 237 result saved.
Section 218 Finished percentage: 75.966941
Section 257 Finished percentage: 31.243129
Section 244 Finished percentage: 82.758961
Section 218 Finished percentage: 77.276716
Section 257 Finished percentage: 32.400282
Section 244 Finished percentage: 83.994170
Section 218 Finished percentage: 78.586491
Section 244 Finished percentage: 85.229378
Section 257 Finished percentage: 33.557435
Section 218 Finished percentage: 79.896266
Section 244 Finished percentage: 86.464587
Section 257 Finished percentage: 34.714588
Section 218 Finished percentage: 81.206041
Section 244 Finished percentage: 87.699795
Section 257 Finished percentage: 35.871741
Section 218 Finished percentage: 82.515816
Section 244 Finished percentage: 88.935003
Section 257 Finished percentage: 37.028894
Section 244 Finished percentage: 90.170212
Section 218 Finished percentage: 83.825590
Section 244 Finished percentage: 91.405420
Section 218 Finished percent

In [3]:
upload_to_s3('/shared/blob_matching_atlas',is_dir=True)

aws s3 cp --recursive "/shared/blob_matching_atlas" "s3://mousebrainatlas-data/blob_matching_atlas"


# Randomly selected False-matched blobs 

In [10]:
def fun_collect_blobs_RandomFalseMatched(section, scan_parameters):
    scan_range = scan_parameters['scan_section_range']
    im0max = scan_parameters['im0max']
    im1max = scan_parameters['im1max']
    prop = scan_parameters['prop']
    prop_for_comparison = scan_parameters['prop_for_comparison']
    compare_weight = scan_parameters['compare_weight']
    o_simil_threshold = scan_parameters['similarity_threshold']
    o_false_simil_threshold = scan_parameters['false_similarity_threshold'] # Option. Default is equal to o_simil_threshold
    o_fix_scan_size = scan_parameters['o_fix_scan_size']
    o_scan_size_coeff = scan_parameters['scan_size_coeff']
    o_scan_size = scan_parameters['scan_size']  
    secList = scan_parameters['secList'] 
    stack = scan_parameters['stack']
    dataFolderName = scan_parameters['dataFolderName']
    # data_typical_blobs = {}
    # data_matched_paris = {}
    # data_false_typical_blobs = {}
    # data_false_matched_pairs = {}
    # data_blob_prop_dic = {}
    # data_blob_idx_selection = {}
#     section = 154
    # for section in secList:
    typical_blobs = []
    matched_paris = []
    false_typical_blobs = []
    false_matched_paris = []
    cell_centroids = {}
    cell_numbers = {}
    cell_global_coord = {}
    im_blob_prop = {}
    im_label = {}
    im_BW = {}
    sec_load_data_list = range(section - scan_range, section + scan_range + 1)
    scan_section = list(sec_load_data_list)
    scan_section.remove(section)
    ### loading data and reconstructed labeled images ###
    tempList = list(sec_load_data_list)
    for tempSec in tempList:
        if tempSec in secList:
            cell_global_coord[tempSec] = load_cell_data('coords', stack=stack, sec=tempSec)
            _, temp_im_blob_prop, _ = fun_reconstruct_labeled_image(cell_global_coord[tempSec],crop_range= scan_parameters['crop_range_mxmx'], 
                                                                        oriImL0=scan_parameters['oriImL0'],oriImL1=scan_parameters['oriImL1'])
#             im_label[tempSec] = temp_im_label
#             im_BW[tempSec] = temp_im_label > 0
            im_blob_prop[tempSec] = np.array(temp_im_blob_prop)
        else:
            sys.stderr.write('Warning: missing section %d'%tempSec)
            scan_section.remove(tempSec)
            sec_load_data_list.remove(tempSec)

    ### Start getting region properties of each blob    ###
    blob_prop_dic = fun_regionprops_dic(im_blob_prop=im_blob_prop,scan_parameters=scan_parameters)
    # data_blob_prop_dic[section] = blob_prop_dic[section] # Save data
    blob_idx_selection = {tempSec : np.logical_and.reduce(np.row_stack(
        (
        #      blob_prop_dic[tempSec]['compactness']<2.5,
         blob_prop_dic[tempSec]['area']<3000,
         blob_prop_dic[tempSec]['area']>200))) for tempSec in sec_load_data_list}  

    # data_blob_idx_selection[section] = blob_idx_selection[section] # Save data

    n_blobs = {tempSec: len(im_blob_prop[tempSec]) for tempSec in im_blob_prop.keys()}
    secList_in_BlobPropDic = im_blob_prop.keys()
    if set(scan_section).issubset(set(secList_in_BlobPropDic)):
        pass
    else:
        print('Warrning: Scaned section(s) not included in input im_blob_prop')



    ### Start scanning ###
    for blobID in range(n_blobs[section]):
        if (blobID % 1000 == 0):
            print('Section %d Finished percentage: %f'%(section, (float(blobID)*100 / n_blobs[section]) ))

        temp_curr_blob_props = {}
        for tempProp in prop:
            temp_curr_blob_props[tempProp] = blob_prop_dic[section][tempProp][blobID]
        if not blob_idx_selection[section][blobID]:
    #         print('Blob %d eliminated'%blobID)
            continue

        tempB1_idx_loc = temp_curr_blob_props['centroid']
        if o_fix_scan_size:
            temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max,o_form='2D')
            temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max)
        else:
            temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max,o_form='2D')
            temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max)


        for tempSec in scan_section:
    #         print('Blbo %d left. Start scanning'%blobID)
            if tempSec not in secList_in_BlobPropDic:
                continue

            # Find blobs at the nearby location in the scaned section
            tempBlobInside = fun_blobs_in_polygen(blob_prop_dic[tempSec]['centroid'],temp_next_sec_range,coor_order='cr')
            tempBlobInsideIndex = np.where(tempBlobInside)[0]
            temp_num_blob = len(tempBlobInsideIndex)

            tempBlobOutside = fun_blobs_out_polygen(blob_prop_dic[tempSec]['centroid'], temp_next_sec_range, coor_order='cr', margin=scan_parameters['scan_outside_margin'])
            tempBlobOutsideIndex = np.where(tempBlobOutside)[0]
            tempBlobOutsideIndex = np.random.choice(tempBlobOutsideIndex, size=scan_parameters['false_match_sample_num'], replace=False)
            temp_num_outblob = len(tempBlobOutsideIndex)

            ### Typical blobs###

            if temp_num_blob:
                temp_sim = {}
                for temp_prop in prop_for_comparison:
                          temp_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobInside],distance_type=temp_prop))
                temp_sim_matrix = np.column_stack((temp_sim[temp_prop] for temp_prop in prop_for_comparison))

                #### Blob comparison ####
                temp_weighted_sim = np.dot(temp_sim_matrix,compare_weight)
                temp_compare_result = temp_weighted_sim > o_simil_threshold
                if any(temp_compare_result.tolist()):
                    typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                    matched_paris.append([section,blobID,
                                          im_blob_prop[section][blobID],
                                          tempSec,
                                          tempBlobInsideIndex[temp_compare_result],
                                          im_blob_prop[tempSec][tempBlobInsideIndex[temp_compare_result]],
                                          temp_sim_matrix[temp_compare_result,:]])
            else:
                pass

            ### False-typical blobs###

                
            if temp_num_outblob:
                false_typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                false_matched_paris.append([section,blobID,im_blob_prop[section][blobID],
                                                tempSec, 
                                                tempBlobOutsideIndex, 
                                                im_blob_prop[tempSec][tempBlobOutsideIndex],
                                                ])
                                        

    tempFp = get_typical_cell_data_filepath(what='scan_parameters',stack=stack,sec=section,dataFolderName=dataFolderName)
    create_if_not_exists(os.path.dirname(tempFp))
    save_pickle(scan_parameters,tempFp)
    fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='typical',dataFolderName=dataFolderName)

    fun_save_matched_blobs_info(matched_paris=matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='matched',dataFolderName=dataFolderName)
    fun_save_typical_blobs_info(typical_blobs=false_typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_typical',dataFolderName=dataFolderName)
    fun_save_matched_blobs_info(matched_paris=false_matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_matched',dataFolderName=dataFolderName)
    print('Section %d result saved.'% section)
    return 0
# data_typical_blobs[section] = typical_blobs
# data_matched_paris[section] = matched_paris
# data_false_typical_blobs[section] = false_typical_blobs
# data_false_matched_pairs[section] = false_matched_paris   

In [11]:
from multiprocess import Pool

In [12]:
scan_parameters['dataFolderName'] = 'Area_NoCoors_RandomFalseMatched'
scan_parameters['scan_outside_margin'] = 3000
scan_parameters['false_match_sample_num'] = 10

In [13]:
fun_collect_blobs_RandomFalseMatched(section=152, scan_parameters=scan_parameters)

Section 152 Finished percentage: 0.000000
Section 152 Finished percentage: 1.458683
Section 152 Finished percentage: 2.917366
Section 152 Finished percentage: 4.376048
Section 152 Finished percentage: 5.834731
Section 152 Finished percentage: 7.293414
Section 152 Finished percentage: 8.752097
Section 152 Finished percentage: 10.210780
Section 152 Finished percentage: 11.669462
Section 152 Finished percentage: 13.128145
Section 152 Finished percentage: 14.586828
Section 152 Finished percentage: 16.045511
Section 152 Finished percentage: 17.504194
Section 152 Finished percentage: 18.962877
Section 152 Finished percentage: 20.421559
Section 152 Finished percentage: 21.880242
Section 152 Finished percentage: 23.338925
Section 152 Finished percentage: 24.797608
Section 152 Finished percentage: 26.256291
Section 152 Finished percentage: 27.714973
Section 152 Finished percentage: 29.173656
Section 152 Finished percentage: 30.632339
Section 152 Finished percentage: 32.091022
Section 152 Finish

KeyboardInterrupt: 

In [ ]:
pool = Pool(2)
pool.map(lambda sec: fun_collect_blobs_RandomFalseMatched(section=sec, scan_parameters=scan_parameters), secList)

Section 92 Finished percentage: 0.000000
Section 92 Finished percentage: 1.916847
Section 92 Finished percentage: 3.833694
Section 92 Finished percentage: 5.750542
Section 92 Finished percentage: 7.667389
Section 92 Finished percentage: 9.584236
Section 128 Finished percentage: 0.000000
Section 92 Finished percentage: 11.501083
Section 128 Finished percentage: 1.946245
Section 92 Finished percentage: 13.417930
Section 128 Finished percentage: 3.892489
Section 92 Finished percentage: 15.334777
Section 92 Finished percentage: 17.251625
Section 128 Finished percentage: 5.838734
Section 92 Finished percentage: 19.168472
Section 92 Finished percentage: 21.085319
Section 128 Finished percentage: 7.784979
Section 92 Finished percentage: 23.002166
Section 92 Finished percentage: 24.919013
Section 128 Finished percentage: 9.731224
Section 92 Finished percentage: 26.835860
Section 92 Finished percentage: 28.752708
Section 128 Finished percentage: 11.677468
Section 92 Finished percentage: 30.6695

Section 129 Finished percentage: 54.521663
Section 129 Finished percentage: 56.339052
Section 93 Finished percentage: 86.438913
Section 129 Finished percentage: 58.156441
Section 129 Finished percentage: 59.973830
Section 93 Finished percentage: 88.403434
Section 129 Finished percentage: 61.791218
Section 129 Finished percentage: 63.608607
Section 93 Finished percentage: 90.367955
Section 129 Finished percentage: 65.425996
Section 129 Finished percentage: 67.243385
Section 129 Finished percentage: 69.060773
Section 129 Finished percentage: 70.878162
Section 93 Finished percentage: 92.332475
Section 129 Finished percentage: 72.695551
Section 129 Finished percentage: 74.512940
Section 129 Finished percentage: 76.330329
Section 93 Finished percentage: 94.296996
Section 129 Finished percentage: 78.147717
Section 129 Finished percentage: 79.965106
Section 129 Finished percentage: 81.782495
Section 93 Finished percentage: 96.261517
Section 129 Finished percentage: 83.599884
Section 129 Finis

Section 131 Finished percentage: 27.659367
Section 131 Finished percentage: 29.286388
Section 131 Finished percentage: 30.913410
Section 131 Finished percentage: 32.540431
Section 131 Finished percentage: 34.167453
Section 131 Finished percentage: 35.794475
Section 131 Finished percentage: 37.421496
Section 131 Finished percentage: 39.048518
Section 131 Finished percentage: 40.675539
Section 131 Finished percentage: 42.302561
Section 131 Finished percentage: 43.929583
Section 94 result saved.
Section 131 Finished percentage: 45.556604
Section 131 Finished percentage: 47.183626
Section 131 Finished percentage: 48.810647
Section 131 Finished percentage: 50.437669
Section 131 Finished percentage: 52.064690
Section 131 Finished percentage: 53.691712
Section 131 Finished percentage: 55.318734
rm -rf "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-N17-2015.07.30-17.07.32_MD589_2_0050/MD589-N17-2015.07.30-17.07.32_MD589_2_0050_blobCoords.hdf" && mkdir -p "/shared/CSHL_cells_v2/detected_cell

aws s3 cp "s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD589/MD589-IHC17-2015.07.30-22.23.59_MD589_2_0050/MD589-IHC17-2015.07.30-22.23.59_MD589_2_0050_blobCoords.hdf" "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC17-2015.07.30-22.23.59_MD589_2_0050/MD589-IHC17-2015.07.30-22.23.59_MD589_2_0050_blobCoords.hdf"
Section 96 Finished percentage: 0.000000
Section 96 Finished percentage: 1.938698
Section 96 Finished percentage: 3.877397
Section 96 Finished percentage: 5.816095
Section 96 Finished percentage: 7.754793
Section 96 Finished percentage: 9.693492
Section 96 Finished percentage: 11.632190
Section 96 Finished percentage: 13.570889
Section 96 Finished percentage: 15.509587
Section 96 Finished percentage: 17.448285
Section 96 Finished percentage: 19.386984
Section 96 Finished percentage: 21.325682
Section 96 Finished percentage: 23.264380
Section 96 Finished percentage: 25.203079
Section 96 Finished percentage: 27.141777
Section 96 Finished percentage: 29.080475
Sectio

Section 97 Finished percentage: 78.855469
Section 97 Finished percentage: 80.732980
Section 97 Finished percentage: 82.610492
Section 97 Finished percentage: 84.488003
Section 97 Finished percentage: 86.365514
Section 97 Finished percentage: 88.243025
Section 97 Finished percentage: 90.120536
Section 97 Finished percentage: 91.998047
Section 97 Finished percentage: 93.875559
Section 97 Finished percentage: 95.753070
Section 97 Finished percentage: 97.630581
Section 97 Finished percentage: 99.508092
Section 133 result saved.
rm -rf "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC23-2015.07.30-22.48.41_MD589_3_0069/MD589-IHC23-2015.07.30-22.48.41_MD589_3_0069_blobCoords.hdf" && mkdir -p "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC23-2015.07.30-22.48.41_MD589_3_0069"
aws s3 cp "s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD589/MD589-IHC23-2015.07.30-22.48.41_MD589_3_0069/MD589-IHC23-2015.07.30-22.48.41_MD589_3_0069_blobCoords.hdf" "/shared/CSHL_cells_v2/detected_c

Section 135 Finished percentage: 41.778478
Section 135 Finished percentage: 43.385342
Section 135 Finished percentage: 44.992207
Section 135 Finished percentage: 46.599071
Section 135 Finished percentage: 48.205936
Section 135 Finished percentage: 49.812800
Section 135 Finished percentage: 51.419665
Section 135 Finished percentage: 53.026529
Section 135 Finished percentage: 54.633394
Section 135 Finished percentage: 56.240258
Section 135 Finished percentage: 57.847123
Section 135 Finished percentage: 59.453987
Section 135 Finished percentage: 61.060852
Section 135 Finished percentage: 62.667716
Section 135 Finished percentage: 64.274581
Section 135 Finished percentage: 65.881446
Section 135 Finished percentage: 67.488310
Section 135 Finished percentage: 69.095175
Section 135 Finished percentage: 70.702039
Section 135 Finished percentage: 72.308904
Section 135 Finished percentage: 73.915768
Section 135 Finished percentage: 75.522633
Section 135 Finished percentage: 77.129497
Section 135

Section 100 Finished percentage: 0.000000
Section 100 Finished percentage: 1.824218
Section 100 Finished percentage: 3.648437
Section 100 Finished percentage: 5.472655
Section 100 Finished percentage: 7.296873
Section 100 Finished percentage: 9.121092
Section 100 Finished percentage: 10.945310
Section 136 result saved.
Section 100 Finished percentage: 12.769528
Section 100 Finished percentage: 14.593747
Section 100 Finished percentage: 16.417965
Section 100 Finished percentage: 18.242183
Section 100 Finished percentage: 20.066402
Section 100 Finished percentage: 21.890620
Section 100 Finished percentage: 23.714838
Section 100 Finished percentage: 25.539057
Section 100 Finished percentage: 27.363275
Section 100 Finished percentage: 29.187493
Section 100 Finished percentage: 31.011711
Section 100 Finished percentage: 32.835930
Section 100 Finished percentage: 34.660148
Section 100 Finished percentage: 36.484366
Section 137 Finished percentage: 0.000000
Section 100 Finished percentage: 38

Section 138 Finished percentage: 24.272630
Section 101 Finished percentage: 70.336857
Section 138 Finished percentage: 25.890806
Section 101 Finished percentage: 72.237853
Section 138 Finished percentage: 27.508981
Section 101 Finished percentage: 74.138849
Section 138 Finished percentage: 29.127156
Section 101 Finished percentage: 76.039845
Section 138 Finished percentage: 30.745332
Section 138 Finished percentage: 32.363507
Section 101 Finished percentage: 77.940841
Section 138 Finished percentage: 33.981682
Section 101 Finished percentage: 79.841837
Section 138 Finished percentage: 35.599858
Section 138 Finished percentage: 37.218033
Section 101 Finished percentage: 81.742833
Section 138 Finished percentage: 38.836208
Section 138 Finished percentage: 40.454384
Section 101 Finished percentage: 83.643829
Section 138 Finished percentage: 42.072559
Section 138 Finished percentage: 43.690734
Section 138 Finished percentage: 45.308910
Section 101 Finished percentage: 85.544825
Section 138

Section 139 Finished percentage: 91.884522
Section 102 Finished percentage: 93.800004
Section 139 Finished percentage: 93.366530
Section 139 Finished percentage: 94.848539
Section 139 Finished percentage: 96.330547
Section 102 Finished percentage: 95.639219
Section 139 Finished percentage: 97.812556
Section 139 Finished percentage: 99.294564
Section 102 Finished percentage: 97.478435
Section 102 Finished percentage: 99.317651
Section 139 result saved.
rm -rf "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC24-2015.07.30-22.52.55_MD589_3_0072/MD589-IHC24-2015.07.30-22.52.55_MD589_3_0072_blobCoords.hdf" && mkdir -p "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC24-2015.07.30-22.52.55_MD589_3_0072"
aws s3 cp "s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD589/MD589-IHC24-2015.07.30-22.52.55_MD589_3_0072/MD589-IHC24-2015.07.30-22.52.55_MD589_3_0072_blobCoords.hdf" "/shared/CSHL_cells_v2/detected_cells/MD589/MD589-IHC24-2015.07.30-22.52.55_MD589_3_0072/MD589-IHC24-2015.0

Section 141 Finished percentage: 46.586262
Section 141 Finished percentage: 48.311679
Section 141 Finished percentage: 50.037096
Section 141 Finished percentage: 51.762514
Section 141 Finished percentage: 53.487931
Section 141 Finished percentage: 55.213348
Section 141 Finished percentage: 56.938765
Section 141 Finished percentage: 58.664182
Section 141 Finished percentage: 60.389599
Section 141 Finished percentage: 62.115016
Section 141 Finished percentage: 63.840433
Section 141 Finished percentage: 65.565851
Section 141 Finished percentage: 67.291268
Section 141 Finished percentage: 69.016685
Section 141 Finished percentage: 70.742102
Section 141 Finished percentage: 72.467519
Section 141 Finished percentage: 74.192936
Section 141 Finished percentage: 75.918353
Section 141 Finished percentage: 77.643770
Section 141 Finished percentage: 79.369188
Section 141 Finished percentage: 81.094605
Section 141 Finished percentage: 82.820022
Section 141 Finished percentage: 84.545439
Section 141

# For debuging

In [13]:
scan_range = scan_parameters['scan_section_range']
im0max = scan_parameters['im0max']
im1max = scan_parameters['im1max']
prop = scan_parameters['prop']
prop_for_comparison = scan_parameters['prop_for_comparison']
compare_weight = scan_parameters['compare_weight']
o_simil_threshold = scan_parameters['similarity_threshold']
o_fix_scan_size = scan_parameters['o_fix_scan_size']
o_scan_size_coeff = scan_parameters['scan_size_coeff']
o_scan_size = scan_parameters['scan_size']  
secList = scan_parameters['secList'] 
stack = scan_parameters['stack']

data_typical_blobs = {}
data_matched_paris = {}
data_false_typical_blobs = {}
data_false_matched_pairs = {}
data_blob_prop_dic = {}
data_blob_idx_selection = {}
section = 154
# for section in secList:
typical_blobs = []
matched_paris = []
false_typical_blobs = []
false_matched_paris = []
cell_centroids = {}
cell_numbers = {}
cell_global_coord = {}
im_blob_prop = {}
im_label_ori = {}
im_label = {}
im_BW = {}
sec_load_data_list = range(section - scan_range, section + scan_range + 1)
scan_section = list(sec_load_data_list)
scan_section.remove(section)
### loading data and reconstructed labeled images ###
tempList = list(sec_load_data_list)
for tempSec in tempList:
    if tempSec in secList:
        cell_global_coord[tempSec] = load_cell_data('coords', stack=stack, sec=tempSec)
        temp_im_label, temp_im_blob_prop, _ = fun_reconstruct_labeled_image(cell_global_coord[tempSec],crop_range= scan_parameters['crop_range_mxmx'], 
                                                                    oriImL0=scan_parameters['oriImL0'],oriImL1=scan_parameters['oriImL1'])
        im_label[tempSec] = temp_im_label
        im_BW[tempSec] = temp_im_label > 0
        im_blob_prop[tempSec] = np.array(temp_im_blob_prop)
    else:
        sys.stderr.write('Warning: missing section %d'%tempSec)
        scan_section.remove(tempSec)
        sec_load_data_list.remove(tempSec)

### Start getting region properties of each blob    ###
blob_prop_dic = fun_regionprops_dic(im_blob_prop=im_blob_prop,scan_parameters=scan_parameters)
data_blob_prop_dic[section] = blob_prop_dic[section] # Save data
blob_idx_selection = {tempSec : np.logical_and.reduce(np.row_stack(
    (
#      blob_prop_dic[tempSec]['compactness']<2.5,
     blob_prop_dic[tempSec]['area']<3000,
     blob_prop_dic[tempSec]['area']>200))) for tempSec in sec_load_data_list}  

data_blob_idx_selection[section] = blob_idx_selection[section] # Save data

n_blobs = {tempSec: len(im_blob_prop[tempSec]) for tempSec in im_blob_prop.keys()}
secList_in_BlobPropDic = im_blob_prop.keys()
if set(scan_section).issubset(set(secList_in_BlobPropDic)):
    pass
else:
    print('Warrning: Scaned section(s) not included in input im_blob_prop')



### Start scanning ###
for blobID in range(n_blobs[section]):
    if (blobID % 1000 == 0):
        print('Section %d Finished percentage: %f'%(section, (float(blobID)*100 / n_blobs[section]) )) 

    temp_curr_blob_props = {}
    for tempProp in prop:
        temp_curr_blob_props[tempProp] = blob_prop_dic[section][tempProp][blobID]
    if not blob_idx_selection[section][blobID]:
#         print('Blob %d eliminated'%blobID)
        continue

    tempB1_idx_loc = temp_curr_blob_props['centroid']
    if o_fix_scan_size:
        temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max,o_form='2D')
        temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max)
    else:
        temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max,o_form='2D')
        temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max)


    for tempSec in scan_section:
#         print('Blbo %d left. Start scanning'%blobID)
        if tempSec not in secList_in_BlobPropDic:
            continue

        # Find blobs at the nearby location in the scaned section
        tempBlobInside = fun_blobs_in_polygen(blob_prop_dic[tempSec]['centroid'],temp_next_sec_range,coor_order='cr')
        tempBlobOutside = fun_blobs_out_polygen(blob_prop_dic[tempSec]['centroid'], temp_next_sec_range, coor_order='cr', margin=scan_parameters['scan_outside_margin'])
        tempBlobInsideIndex = np.where(tempBlobInside)[0]
        temp_num_blob = len(tempBlobInsideIndex)
        tempBlobOutsideIndex = np.where(tempBlobOutside)[0]
        tempBlobOutsideIndex = np.random.choice(tempBlobOutsideIndex, size=scan_parameters['false_match_sample_num'], replace=False)

        temp_num_outblob = len(tempBlobOutsideIndex)

        ### Typical blobs###

        if temp_num_blob:
            temp_sim = {}
            for temp_prop in prop_for_comparison:
                      temp_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobInside],distance_type=temp_prop))
            temp_sim_matrix = np.column_stack((temp_sim[temp_prop] for temp_prop in prop_for_comparison))

            #### Blob comparison ####
            temp_weighted_sim = np.dot(temp_sim_matrix,compare_weight)
            temp_compare_result = temp_weighted_sim > o_simil_threshold
            if any(temp_compare_result.tolist()):
                typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                matched_paris.append([section,blobID,
                                      im_blob_prop[section][blobID],
                                      tempSec,
                                      tempBlobInsideIndex[temp_compare_result],
                                      im_blob_prop[tempSec][tempBlobInsideIndex[temp_compare_result]],
                                      temp_sim_matrix[temp_compare_result,:]])
        else:
            pass

        ### False-typical blobs###
        if temp_num_outblob:
            temp_false_sim = {}
            for temp_prop in prop_for_comparison:
                      temp_false_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobOutsideIndex],distance_type=temp_prop))
            temp_false_sim_matrix = np.column_stack((temp_false_sim[temp_prop] for temp_prop in prop_for_comparison))

            #### Blob comparison ####
            temp_weighted_false_sim = np.dot(temp_false_sim_matrix,compare_weight)
            temp_false_compare_result = temp_weighted_false_sim > o_simil_threshold
            if any(temp_false_compare_result.tolist()):
                false_typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                false_matched_paris.append([section,blobID,im_blob_prop[section][blobID],
                                            tempSec, 
                                            tempBlobOutsideIndex[temp_false_compare_result], 
                                            im_blob_prop[tempSec][tempBlobOutsideIndex[temp_false_compare_result]],
                                            temp_false_sim_matrix[temp_false_compare_result,:]])
        else:
            pass
fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='typical')
fun_save_matched_blobs_info(matched_paris=matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='matched')
fun_save_typical_blobs_info(typical_blobs=false_typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='false_typical')
fun_save_matched_blobs_info(matched_paris=false_matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='false_matched')

# data_typical_blobs[section] = typical_blobs
# data_matched_paris[section] = matched_paris
# data_false_typical_blobs[section] = false_typical_blobs
# data_false_matched_pairs[section] = false_matched_paris   

Section 154 Finished percentage: 0.000000
Section 154 Finished percentage: 1.460814
Section 154 Finished percentage: 2.921627
Section 154 Finished percentage: 4.382441
Section 154 Finished percentage: 5.843255
Section 154 Finished percentage: 7.304068
Section 154 Finished percentage: 8.764882
Section 154 Finished percentage: 10.225696
Section 154 Finished percentage: 11.686509
Section 154 Finished percentage: 13.147323
Section 154 Finished percentage: 14.608137
Section 154 Finished percentage: 16.068950
Section 154 Finished percentage: 17.529764
Section 154 Finished percentage: 18.990578
Section 154 Finished percentage: 20.451391
Section 154 Finished percentage: 21.912205
Section 154 Finished percentage: 23.373019
Section 154 Finished percentage: 24.833832
Section 154 Finished percentage: 26.294646
Section 154 Finished percentage: 27.755460
Section 154 Finished percentage: 29.216273
Section 154 Finished percentage: 30.677087
Section 154 Finished percentage: 32.137901
Section 154 Finish

In [30]:
# def fun_collect_blobs(section, scan_parameters, collectFlaseMatched=False, saveMatched=True):
section = 92
collectFlaseMatched=False
saveMatched=True
scan_range = scan_parameters['scan_section_range']
im0max = scan_parameters['im0max']
im1max = scan_parameters['im1max']
prop = scan_parameters['prop']
prop_for_comparison = scan_parameters['prop_for_comparison']
compare_weight = scan_parameters['compare_weight']
o_simil_threshold = scan_parameters['similarity_threshold']
o_false_simil_threshold = scan_parameters['false_similarity_threshold'] # Option. Default is equal to o_simil_threshold
o_fix_scan_size = scan_parameters['o_fix_scan_size']
o_scan_size_coeff = scan_parameters['scan_size_coeff']
o_scan_size = scan_parameters['scan_size']  
secList = scan_parameters['secList'] 
stack = scan_parameters['stack']
dataFolderName = scan_parameters['dataFolderName']
# data_typical_blobs = {}
# data_matched_paris = {}
# data_false_typical_blobs = {}
# data_false_matched_pairs = {}
# data_blob_prop_dic = {}
# data_blob_idx_selection = {}
#     section = 154
# for section in secList:
typical_blobs = []
matched_paris = []
false_typical_blobs = []
false_matched_paris = []
cell_centroids = {}
cell_numbers = {}
cell_global_coord = {}
im_blob_prop = {}
im_label = {}
im_BW = {}
sec_load_data_list = range(section - scan_range, section + scan_range + 1)
scan_section = list(sec_load_data_list)
scan_section.remove(section)
### loading data and reconstructed labeled images ###
tempList = list(sec_load_data_list)
for tempSec in tempList:
    if tempSec in secList:
        cell_global_coord[tempSec] = load_cell_data('coords', stack=stack, sec=tempSec)
        _, temp_im_blob_prop, _ = fun_reconstruct_labeled_image(cell_global_coord[tempSec],crop_range= scan_parameters['crop_range_mxmx'], 
                                                                    oriImL0=scan_parameters['oriImL0'],oriImL1=scan_parameters['oriImL1'])
#             im_label[tempSec] = temp_im_label
#             im_BW[tempSec] = temp_im_label > 0
        im_blob_prop[tempSec] = np.array(temp_im_blob_prop)
    else:
        sys.stderr.write('Warning: missing section %d'%tempSec)
        scan_section.remove(tempSec)
        sec_load_data_list.remove(tempSec)

### Start getting region properties of each blob    ###
blob_prop_dic = fun_regionprops_dic(im_blob_prop=im_blob_prop,scan_parameters=scan_parameters)
# data_blob_prop_dic[section] = blob_prop_dic[section] # Save data
blob_idx_selection = {tempSec : np.logical_and.reduce(np.row_stack(
    (
    #      blob_prop_dic[tempSec]['compactness']<2.5,
     blob_prop_dic[tempSec]['area']<3000,
     blob_prop_dic[tempSec]['area']>200))) for tempSec in sec_load_data_list}  

# data_blob_idx_selection[section] = blob_idx_selection[section] # Save data

n_blobs = {tempSec: len(im_blob_prop[tempSec]) for tempSec in im_blob_prop.keys()}
secList_in_BlobPropDic = im_blob_prop.keys()
if set(scan_section).issubset(set(secList_in_BlobPropDic)):
    pass
else:
    print('Warrning: Scaned section(s) not included in input im_blob_prop')

### Start scanning ###
for blobID in range(n_blobs[section]):
    if (blobID % 1000 == 0):
        print('Section %d Finished percentage: %f'%(section, (float(blobID)*100 / n_blobs[section]) ))

    temp_curr_blob_props = {}
    for tempProp in prop:
        temp_curr_blob_props[tempProp] = blob_prop_dic[section][tempProp][blobID]
    if not blob_idx_selection[section][blobID]:
#         print('Blob %d eliminated'%blobID)
        continue

    tempB1_idx_loc = temp_curr_blob_props['centroid']
    if o_fix_scan_size:
        temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max,o_form='2D')
        temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max)
    else:
        temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max,o_form='2D')
        temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max)


    for tempSec in scan_section:
#         print('Blbo %d left. Start scanning'%blobID)
        if tempSec not in secList_in_BlobPropDic:
            continue

        # Find blobs at the nearby location in the scaned section
        tempBlobInside = fun_blobs_in_polygen(blob_prop_dic[tempSec]['centroid'],temp_next_sec_range,coor_order='cr')
        tempBlobInsideIndex = np.where(tempBlobInside)[0]
        temp_num_blob = len(tempBlobInsideIndex)
        if collectFlaseMatched == True:
            tempBlobOutside = fun_blobs_out_polygen(blob_prop_dic[tempSec]['centroid'], temp_next_sec_range, coor_order='cr', margin=scan_parameters['scan_outside_margin'])
            tempBlobOutsideIndex = np.where(tempBlobOutside)[0]
            tempBlobOutsideIndex = np.random.choice(tempBlobOutsideIndex, size=scan_parameters['false_match_sample_num'], replace=False)
            temp_num_outblob = len(tempBlobOutsideIndex)

        ### Typical blobs###

        if temp_num_blob:
            temp_sim = {}
            for temp_prop in prop_for_comparison:
                      temp_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobInside],distance_type=temp_prop))
            temp_sim_matrix = np.column_stack((temp_sim[temp_prop] for temp_prop in prop_for_comparison))

            #### Blob comparison ####
            temp_weighted_sim = np.dot(temp_sim_matrix,compare_weight)
            temp_compare_result = temp_weighted_sim > o_simil_threshold
            if any(temp_compare_result.tolist()):
                typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                matched_paris.append([section,blobID,
                                      im_blob_prop[section][blobID],
                                      tempSec,
                                      tempBlobInsideIndex[temp_compare_result],
                                      im_blob_prop[tempSec][tempBlobInsideIndex[temp_compare_result]],
                                      temp_sim_matrix[temp_compare_result,:]])
        else:
            pass

        ### False-typical blobs###
        if collectFlaseMatched == True:
            if temp_num_outblob:
                temp_false_sim = {}
                for temp_prop in prop_for_comparison:
                          temp_false_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobOutsideIndex],distance_type=temp_prop))
                temp_false_sim_matrix = np.column_stack((temp_false_sim[temp_prop] for temp_prop in prop_for_comparison))

                #### Blob comparison ####
                temp_weighted_false_sim = np.dot(temp_false_sim_matrix,compare_weight)
                temp_false_compare_result = temp_weighted_false_sim > o_false_simil_threshold
                if any(temp_false_compare_result.tolist()):
                    false_typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                    false_matched_paris.append([section,blobID,im_blob_prop[section][blobID],
                                                tempSec, 
                                                tempBlobOutsideIndex[temp_false_compare_result], 
                                                im_blob_prop[tempSec][tempBlobOutsideIndex[temp_false_compare_result]],
                                                temp_false_sim_matrix[temp_false_compare_result,:]])
            else:
                pass
tempFp = get_typical_cell_data_filepath(what='scan_parameters',stack=stack,sec=section,dataFolderName=dataFolderName)
create_if_not_exists(os.path.dirname(tempFp))
save_pickle(scan_parameters,tempFp)
fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='typical',dataFolderName=dataFolderName)
if saveMatched == True:
    fun_save_matched_blobs_info(matched_paris=matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='matched',dataFolderName=dataFolderName)
if collectFlaseMatched == True:
    fun_save_typical_blobs_info(typical_blobs=false_typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_typical',dataFolderName=dataFolderName)
    fun_save_matched_blobs_info(matched_paris=false_matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_matched',dataFolderName=dataFolderName)
print('Section %d result saved.'% section)
# return 0
# data_typical_blobs[section] = typical_blobs
# data_matched_paris[section] = matched_paris
# data_false_typical_blobs[section] = false_typical_blobs
# data_false_matched_pairs[section] = false_matched_paris   

Section 92 Finished percentage: 0.000000
Section 92 Finished percentage: 1.916847
Section 92 Finished percentage: 3.833694
Section 92 Finished percentage: 5.750542
Section 92 Finished percentage: 7.667389
Section 92 Finished percentage: 9.584236
Section 92 Finished percentage: 11.501083
Section 92 Finished percentage: 13.417930
Section 92 Finished percentage: 15.334777
Section 92 Finished percentage: 17.251625
Section 92 Finished percentage: 19.168472
Section 92 Finished percentage: 21.085319
Section 92 Finished percentage: 23.002166
Section 92 Finished percentage: 24.919013
Section 92 Finished percentage: 26.835860
Section 92 Finished percentage: 28.752708
Section 92 Finished percentage: 30.669555
Section 92 Finished percentage: 32.586402
Section 92 Finished percentage: 34.503249
Section 92 Finished percentage: 36.420096
Section 92 Finished percentage: 38.336943
Section 92 Finished percentage: 40.253791
Section 92 Finished percentage: 42.170638
Section 92 Finished percentage: 44.08748

TypeError: cannot concatenate 'str' and 'NoneType' objects

In [34]:
scan_parameters['prop_to_save']

['moments_hu',
 'centroid',
 'area',
 'eccentricity',
 'equivalent_diameter',
 'compactness',
 'orientation',
 'solidity']

In [37]:
fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='typical',dataFolderName=dataFolderName)

KeyError: 'compactness'